In [3]:
import numpy as np

import ad
from ad import adnumber, jacobian
import ad.admath as am

In [4]:
P_X, P_Y, P_THETA, P_V = adnumber([0.1, 0.1, 0.01, 1])
P = [P_X, P_Y, P_THETA, P_V]
X0 = adnumber([1, 1, 3/2*np.pi, 0])

T = 1
N = 100
D = 4
H = T/N

DIM_X = 4
DIM_U = 2
DIM = DIM_X + DIM_U

C_W = 0.01
C_A = 0.0001

WEIGHT_L = 0.01
WEIGHT_LF = 1

In [5]:
def F(x: "R^4 adarray", u: "R^2 adarray") -> "R^4*":
    x, y, theta, v = x
    w, a = u
    
    f = H*v
    b = f*am.cos(w) + D - am.sqrt(D**2 - f**2 * am.sin(w)**2)
    
#     print("f", f,"b", b)
    x = x + b*am.cos(theta)
    y = y + b*am.sin(theta)
    theta = theta + am.asin(am.sin(w) * f/D)
    v = v + H*a
    
    return [x, y, theta, v]

In [6]:
x = adnumber(1)

In [116]:
type(X0[0])

ad.ADV

In [117]:
?ad.ADV

In [131]:
?ad.ADF.hessian

In [108]:
X = F(X, u)
print(X)

ad(0.07800000000000004) ad(-0.06522249613916564)
[ad(1.133322779553084), ad(2.2967289806742275), ad(4.500218706291709), ad(8.000000000000004)]


In [46]:
jacobian(F(X0, u), [*X0, *u])

[[1.0, 0.0, 0.0, -9.925192341800274e-19, 0.0, 0.0],
 [0.0, 1.0, 0.0, -0.005403023058681398, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0021036774620197415, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.01]]

In [ ]:
x, y, theta, v, w, a = adnumber([x, y, theta, v, w, a])

In [37]:
U = np.ones(2)

In [120]:
u = adnumber([10,20])

In [7]:
def z(x, p):
    return am.sqrt(x**2 + p**2) - p

In [20]:
z(x, p).gradient([x, p])

[0.4472135954999579, -0.10557280900008414]

In [8]:
def L(x, u):
    x, y, theta, v = x
    w, a = u
    return WEIGHT_L *(z(x, P_X) + z(y, P_Y)) + C_W * w**2 + C_A * a**2

In [130]:
L(X0, u).hessian((*X0,*u))

[[9.851853368415764e-05, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 9.851853368415764e-05, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.02, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0002]]

In [9]:
def Lf(x):
    x, y, theta, v = x
    return WEIGHT_LF * (z(x, P_X) + z(y, P_Y) + z(theta, P_THETA) + z(v, P_V))

In [10]:
def J(U: list):
    X = [X0]
    for j in range(N-1):
        X.append(F(X[-1], U[j]))
    return sum([L(X[i], U[i]) for i in range(N-1)]) + Lf(X[-1])

In [162]:
def J(U):
    def j(U, k) -> "L(x_k, u_k)":
        x = X0
        for i in range(k):
            x = F(x, U[i])
        return L(x, U[k])
    def jf(U) -> "Lf(x_N-1)":
        x = X0
        for i in range(N-1):
            x = F(x, U[i])
        return Lf(x)
    return sum(j(U, i) for i in range(N-1)) + jf(U)

In [11]:
def J(U):
    def J_iter(U):
        x = X0
        yield L(x, U[0]) #L(x0, u0)
        for i in range(N-2): #L(x1, u1) ~ L(x_n-2, u_n-2)
            x = F(x, U[i])
            yield L(x, U[i+1])
        x = F(x, U[N-2]) #Lf(x_N-1)
        yield Lf(x) 
    return sum(J_iter(U))

In [9]:
def J_iter(U):
    x = X0
    yield L(x, U[0]) #L(x0, u0)
    for i in range(N-2): #L(x1, u1) ~ L(x_n-2, u_n-2)
        x = F(x, U[i])
        yield L(x, U[i+1])
    x = F(x, U[N-2]) #Lf(x_N-1)
    yield Lf(x)

In [13]:
s = sum(J_iter(U))

f ad(0.0) b ad(0.0)
f ad(7.546778155704818e-05) b ad(6.854797168820426e-05)
f ad(0.0001249091504668728) b ad(9.520791891359792e-05)
f ad(0.00012868244978446523) b ad(0.00012809031979621466)
f ad(0.00018120669945071962) b ad(0.00017942126267023184)
f ad(0.0002350669522925267) b ad(0.00023483408895685898)
f ad(0.0002442794685992048) b ad(0.0002410207631911021)
f ad(0.000319207550931188) b ad(0.00031815389458733634)
f ad(0.0004178291989399293) b ad(0.0003864009337708474)
f ad(0.00045469468990936136) b ad(0.00027618014116459477)
f ad(0.0005163296809038707) b ad(0.0004738404768880322)
f ad(0.0005573905181533394) b ad(0.0005248575245806464)
f ad(0.0005851816259291528) b ad(0.0004291101527362784)
f ad(0.0006424933366673883) b ad(0.0005535570281587354)
f ad(0.0006470182816673928) b ad(0.0006005473226040614)
f ad(0.0007460845955508803) b ad(0.0005920598517792364)
f ad(0.0008282489796134608) b ad(0.0007028326707771981)
f ad(0.0008310635285117595) b ad(0.0006604928605571381)
f ad(0.00091029650990

In [14]:
s

ad(8.520936435853923)

In [ ]:
J(U) = sum(j(U, i) for i in 1:N-2) + jf(U) 

In [10]:
U = adnumber(np.random.rand(99,2))

In [163]:
J(U)

ad(0.0) ad(0.0)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0001912398107832837) ad(0.00018946014277432255)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0001912398107832837) ad(0.00018946014277432255)
ad(0.00027385263128127193) ad(0.00027317397187465176)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.000124

ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0001912398107832837) ad(0.00018946014277432255)
ad(0.00027385263128127193) ad(0.00027317397187465176)
ad(0.00029136186779633616) ad(0.00023840041676193025)
ad(0.00032697003748291965) ad(0.00031254313600115324)
ad(0.0003635005584421969) ad(0.0003609043765075981)
ad(0.0003742403738280272) ad(0.0002907291387059807)
ad(0.0004414743816932139) ad(0.00030892482358213513)
ad(0.0005173691347258667) ad(0.0005125878361678105)
ad(0.0005998707371197622) ad(0.0003975958099515786)
ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128

ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128121777897235)
ad(0.0009483696210650079) ad(0.0009352394570409928)
ad(0.000964522401877701) ad(0.00077376484637659)
ad(0.0009696060200847091) ad(0.0008740491793806093)
ad(0.001007091041105689) ad(0.0009933886938919834)
ad(0.001103014986155494) ad(0.0010991215735640658)
ad(0.001154713737247771) ad(0.001075740098054112)
ad(0.0011692868870566352) ad(0.0011647514411365911)
ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0001912398107832837) ad(0.00018946014277432255)
ad(0.00027385263128127193) ad(0.00027317397187465176)
ad(0.00029136186779633616) ad(0.000238400416761930

ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128121777897235)
ad(0.0009483696210650079) ad(0.0009352394570409928)
ad(0.000964522401877701) ad(0.00077376484637659)
ad(0.0009696060200847091) ad(0.0008740491793806093)
ad(0.001007091041105689) ad(0.0009933886938919834)
ad(0.001103014986155494) ad(0.0010991215735640658)
ad(0.001154713737247771) ad(0.001075740098054112)
ad(0.0011692868870566352) ad(0.0011647514411365911)
ad(0.001243859500139104) ad(0.0009865597896068223)
ad(0.00133818278017552) ad(0.00101419681585746)
ad(0.0014053991361094385) ad(0.0013739966405799109)
ad(0.001453028946701294) ad(0.0009854076039510495)
ad(0.0015107417762935044) ad(0.0009122880834602221)
ad(0.0015190360328142358) ad(0.0014431155250687056)
ad(0.0) ad(0.0)
ad(6.212316

ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128121777897235)
ad(0.0009483696210650079) ad(0.0009352394570409928)
ad(0.000964522401877701) ad(0.00077376484637659)
ad(0.0009696060200847091) ad(0.0008740491793806093)
ad(0.001007091041105689) ad(0.0009933886938919834)
ad(0.001103014986155494) ad(0.0010991215735640658)
ad(0.001154713737247771) ad(0.001075740098054112)
ad(0.0011692868870566352) ad(0.0011647514411365911)
ad(0.001243859500139104) ad(0.0009865597896068223)
ad(0.00133818278017552) ad(0.00101419681585746)
ad(0.0014053991361094385) ad(0.0013739966405799109)
ad(0.001453028946701294) ad(0.0009854076039510495)
ad(0.0015107417762935044) ad(0.0009122880834602221)
ad(0.0015190360328142358) ad(0.0014431155250687056)
ad(0.0015190909583881788) a

ad(0.000964522401877701) ad(0.00077376484637659)
ad(0.0009696060200847091) ad(0.0008740491793806093)
ad(0.001007091041105689) ad(0.0009933886938919834)
ad(0.001103014986155494) ad(0.0010991215735640658)
ad(0.001154713737247771) ad(0.001075740098054112)
ad(0.0011692868870566352) ad(0.0011647514411365911)
ad(0.001243859500139104) ad(0.0009865597896068223)
ad(0.00133818278017552) ad(0.00101419681585746)
ad(0.0014053991361094385) ad(0.0013739966405799109)
ad(0.001453028946701294) ad(0.0009854076039510495)
ad(0.0015107417762935044) ad(0.0009122880834602221)
ad(0.0015190360328142358) ad(0.0014431155250687056)
ad(0.0015190909583881788) ad(0.0010689428383638955)
ad(0.0015626998447075314) ad(0.0015319510481512921)
ad(0.001587385053883232) ad(0.0014311122528161135)
ad(0.0016447145317258609) ad(0.0016033054571806815)
ad(0.0017401377887642758) ad(0.0011523146298126896)
ad(0.0018285456163824997) ad(0.0018236761299821325)
ad(0.0018878299374794794) ad(0.0017446538566709968)
ad(0.001900939133589918) a

ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128121777897235)
ad(0.0009483696210650079) ad(0.0009352394570409928)
ad(0.000964522401877701) ad(0.00077376484637659)
ad(0.0009696060200847091) ad(0.0008740491793806093)
ad(0.001007091041105689) ad(0.0009933886938919834)
ad(0.001103014986155494) ad(0.0010991215735640658)
ad(0.001154713737247771) ad(0.001075740098054112)
ad(0.0011692868870566352) ad(0.0011647514411365911)
ad(0.001243859500139104) ad(0.0009865597896068223)
ad(0.00133818278017552) ad(0.00101419681585746)
ad(0.0014053991361094385) ad(0.0013739966405799109)
ad(0.001453028946701294) ad(0.0009854076039510495)
ad(0.0015107417762935044) ad(0.0009122880834602221)
ad(0.0015190360328142358) ad(0.0014431155250687056)
ad(0.0015190909583881788) a

ad(0.0017401377887642758) ad(0.0011523146298126896)
ad(0.0018285456163824997) ad(0.0018236761299821325)


KeyboardInterrupt: 

In [157]:
J(U)

ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0001912398107832837) ad(0.00018946014277432255)
ad(0.00027385263128127193) ad(0.00027317397187465176)
ad(0.00029136186779633616) ad(0.00023840041676193025)
ad(0.00032697003748291965) ad(0.00031254313600115324)
ad(0.0003635005584421969) ad(0.0003609043765075981)
ad(0.0003742403738280272) ad(0.0002907291387059807)
ad(0.0004414743816932139) ad(0.00030892482358213513)
ad(0.0005173691347258667) ad(0.0005125878361678105)
ad(0.0005998707371197622) ad(0.0003975958099515786)
ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128121777897235)
ad(0.0009483696210650079) ad(0.0009352

ad(8.454533123283177)

In [155]:
J(U).gradient(np.ravel(U))

ad(0.0) ad(0.0)
ad(6.212316809040247e-05) ad(5.519385717267511e-05)
ad(0.00015832290056986824) ad(0.00012489748719657356)
ad(0.00018044245721020584) ad(0.00017147016973817486)
ad(0.0001912398107832837) ad(0.00018946014277432255)
ad(0.00027385263128127193) ad(0.00027317397187465176)
ad(0.00029136186779633616) ad(0.00023840041676193025)
ad(0.00032697003748291965) ad(0.00031254313600115324)
ad(0.0003635005584421969) ad(0.0003609043765075981)
ad(0.0003742403738280272) ad(0.0002907291387059807)
ad(0.0004414743816932139) ad(0.00030892482358213513)
ad(0.0005173691347258667) ad(0.0005125878361678105)
ad(0.0005998707371197622) ad(0.0003975958099515786)
ad(0.00060313548052639) ad(0.0005031115114046969)
ad(0.0006770914803867864) ad(0.0006389425586563746)
ad(0.0007182083969865629) ad(0.0007176552700949301)
ad(0.0007195990115115132) ad(0.0004390305769050862)
ad(0.0007764014764791235) ad(0.0007547277953423581)
ad(0.0008655085654612314) ad(0.0008128121777897235)
ad(0.0009483696210650079) ad(0.0009352

[0.0025523297615749076,
 -0.00679914031201993,
 0.009609776463364283,
 -0.00657169934061113,
 0.01346553946819352,
 -0.00658577119403298,
 0.006496803343493716,
 -0.0064342461261830425,
 0.0028408735738304947,
 -0.006104049494991704,
 0.0015318195570552242,
 -0.006045285065238692,
 0.012644768629338733,
 -0.005871267734547049,
 0.006244639209480468,
 -0.005697412003265977,
 0.002587842534571739,
 -0.005565437684801057,
 0.014157685510106042,
 -0.005326554476885866,
 0.01660189480412535,
 -0.005201176795020196,
 0.0030134645476694883,
 -0.0050082975708252144,
 0.01788085244998733,
 -0.005067997747253773,
 0.012454111512958804,
 -0.0047908099309692965,
 0.00735669966893241,
 -0.0046945624239650464,
 0.0010621030167743762,
 -0.004593753540397608,
 0.019461450807174256,
 -0.004397632104256929,
 0.005301546730717614,
 -0.0041657880861746535,
 0.007801970263800262,
 -0.004021319093350856,
 0.0039047019948991214,
 -0.003984781178619258,
 0.014056196100971034,
 -0.0038839236287778285,
 0.00996